In [1]:
import cx_Oracle
import os
import xlsxwriter
import datetime

now = datetime.datetime.now()
kemarin = now - datetime.timedelta(1)
kemarin_str =  str(kemarin.strftime('%Y%m%d'))
dir = '/home/palugada/Daily/'

os.environ["ORACLE_HOME"] = "/usr/lib/oracle/12.2/client64"
# workbook = xlsxwriter.Workbook('/home/palugada/Daily/Demo.xlsx')
workbook = xlsxwriter.Workbook(dir+"RBT_Daily_Report_"+str(kemarin_str)+".xlsx")
database = cx_Oracle.connect('RBTRPTN/RBTRPTN@192.168.10.66:1522/RBTRPTN')

bold = workbook.add_format({'bold': True, 'bg_color': '#92D050', 'align': 'center', 'valign': 'vcenter', 'border': 1})
bold.set_border()
bold.set_text_wrap()
title = workbook.add_format({'bold': True})
# Create worksheet summary
worksheet = workbook.add_worksheet('SUMMARY')

In [2]:
# A All KPI
cursor = database.cursor()
SUMMARY = ("SELECT calling_date, sms, umb, ivr, http, wap, web, openapi, mobileapps, wcc, csweb, renewal, rbt_req, chg_req, refund_req, \
chg_trans_succ, chg_timeout, decode(chg_req, 0, 0,(((chg_req-chg_timeout)/ chg_req)) * 100) AS chgSuccessRate, chg_succ, \
decode(chg_req, 0, 0,(chg_succ / chg_req)* 100) AS billableRate, chg_insf, chg_subs_locked, chg_other_err, refund_succ, \
decode(refund_req, 0, 0,(refund_succ / refund_req)* 100) AS refundRate, prov_req, deprov_req, prov_succ, prov_error, \
prov_timeout, decode(prov_req, 0, 0,((prov_req-prov_timeout)/ prov_req)* 100) AS provTransactionRate, decode( \
prov_req, 0, 0,(prov_succ / prov_req)* 100) AS provRate, deprov_succ, deprov_err, decode(deprov_req, 0, 0,( \
deprov_succ / deprov_req)* 100) AS deactiveRate, revenue, avg_svc_latency FROM perf_dashboard WHERE \
CALLING_DATE = TO_CHAR(SYSDATE-1,'YYYYMMDD') ORDER BY calling_date DESC")

# Exec query summary
cursor.execute(SUMMARY)
result = cursor.fetchall()

worksheet.write('A1','A All KPI',title)
worksheet.merge_range('B3:B4','Calling Date',bold)
worksheet.merge_range('C3:M3','User InterFace',bold)
worksheet.write('C4','SMS',bold)
worksheet.write('D4', 'UMB',bold)
worksheet.write('E4', 'IVR',bold)
worksheet.write('F4', 'HTTPGW',bold)
worksheet.write('G4', 'WAP',bold)
worksheet.write('H4', 'WEB',bold)
worksheet.write('I4', 'OPENAPI',bold)
worksheet.write('J4', 'MKIOS',bold)
worksheet.write('K4', 'WCC',bold)
worksheet.write('L4', 'CSWEB',bold)
worksheet.write('M4', 'RENEWAL',bold)
worksheet.merge_range('N3:N4','Total Purchase Request Received by RBT Eluon',bold)
worksheet.set_column(1,12, 12)
worksheet.set_column('N:N', 20) # 13
worksheet.merge_range('O3:P3','RBT Eluon Request to INGW',bold)
worksheet.write('O4', 'Charge',bold)
worksheet.write('P4', 'Refund',bold)
worksheet.set_column(14,15, 13)
worksheet.merge_range('Q3:S3','Transaction Response from INGW',bold)
worksheet.write('Q4', 'Success Charging Transaction',bold)
worksheet.write('R4', 'Charging Time Out (802)',bold)
worksheet.write('S4', 'Charging Transaction Success Rate',bold)
worksheet.set_column(16,18, 12)
worksheet.merge_range('T3:X3','Charging Related',bold)
worksheet.write('T4', 'Success Charged/Billable',bold)
worksheet.write('U4', 'Success Charged Rate',bold)
worksheet.write('V4', 'Low Balance',bold)
worksheet.write('W4', 'Subscriber Locked',bold)
worksheet.write('X4', 'Unknown Error Code (502)',bold)
worksheet.set_column(19,23, 12)
worksheet.merge_range('Y3:Z3','Refund Process',bold)
worksheet.write('Y4', 'Success Refund',bold)
worksheet.write('Z4', 'Refund Success Rate',bold)
worksheet.set_column(24,25, 12)
worksheet.merge_range('AA3:AB3','RBT Eluon Request to ProvGW',bold)
worksheet.write('AA4', 'Activation',bold)
worksheet.write('AB4', 'Deactivation',bold)
worksheet.set_column(26,27, 12)
worksheet.merge_range('AC3:AG3','Activation Responses from ProvGW',bold)
worksheet.write('AC4', 'Success',bold)
worksheet.write('AD4', 'Prov Error',bold)
worksheet.write('AE4', 'Time Out',bold)
worksheet.set_column(28,30, 12)
worksheet.write('AF4', 'Provisioning Transaction Success Rate',bold)
worksheet.set_column('AF:AF', 15)
worksheet.set_row(3, 50)
worksheet.write('AG4', 'Provisioning Success Rate',bold)
worksheet.set_column('AG:AG', 13)
worksheet.merge_range('AH3:AJ3','Deactivation Responses from ProvGW',bold)
worksheet.write('AH4', 'Success',bold)
worksheet.write('AI4', 'Failed',bold)
worksheet.write('AJ4', 'Success Rate',bold)
worksheet.set_column(33,34, 12)
worksheet.merge_range('AK3:AK4','Revenue',bold)
worksheet.set_column('AK:AK', 18) # 35
worksheet.merge_range('AL3:AL4','Average Purchase Service Latency (ms)',bold)
worksheet.set_column('AL:AL', 20) # 36

# for row in result:
#     print(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17], row[18], row[19], row[20], row[21], row[22], row[23], row[24], row[25], row[26], row[27], row[28], row[29], row[30], row[31], row[32], row[33], row[34], row[35], row[36])
    
for r, row in enumerate(result, start=4):
    for c, col in enumerate(row):
        content = workbook.add_format({'font_color': 'black', 'num_format': '#,##0', 'align': 'center', 'valign': 'vcenter', 'border': 1})
        content.set_border()
        date_format = workbook.add_format({'font_color': 'black', 'num_format': 'yyyymmdd', 'align': 'center', 'valign': 'vcenter', 'border': 1})
        date_format.set_border()
        worksheet.write(r,1, row[0], date_format)
        worksheet.write(r,2, row[1], content)
        worksheet.write(r,3, row[2], content)
        worksheet.write(r,4, row[3], content)
        worksheet.write(r,5, row[4], content)
        worksheet.write(r,6, row[5], content)
        worksheet.write(r,7, row[6], content)
        worksheet.write(r,8, row[7], content)
        worksheet.write(r,9, row[8], content)
        worksheet.write(r,10, row[9], content)
        worksheet.write(r,11, row[10], content)
        worksheet.write(r,12, row[11], content)
        worksheet.write(r,13, row[12], content)
        worksheet.write(r,14, row[13], content)
        worksheet.write(r,15, row[14], content)
        worksheet.write(r,16, row[15], content)
        worksheet.write(r,17, row[16], content)
        worksheet.write(r,18, row[17], content)
        worksheet.write(r,19, row[18], content)
        worksheet.write(r,20, row[19], content)
        worksheet.write(r,21, row[20], content)
        worksheet.write(r,22, row[21], content)
        worksheet.write(r,23, row[22], content)
        worksheet.write(r,24, row[23], content)
        worksheet.write(r,25, row[24], content)
        worksheet.write(r,26, row[25], content)
        worksheet.write(r,27, row[26], content)
        worksheet.write(r,28, row[27], content)
        worksheet.write(r,29, row[28], content)
        worksheet.write(r,30, row[29], content)
        worksheet.write(r,31, row[30], content)
        worksheet.write(r,32, row[31], content)
        worksheet.write(r,33, row[32], content)
        worksheet.write(r,34, row[33], content)
        worksheet.write(r,35, row[34], content)
        worksheet.write(r,36, row[35], content)
        worksheet.write(r,37, row[36], content)

cursor.close()
workbook.close()
database.close()